In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [4]:
df=pd.read_csv("/kaggle/input/lacuna-malaria-detection-dataset/Train.csv")
df=df.drop(columns=["Image_ID"])

In [8]:
df=pd.get_dummies(df).astype(int)
df

,confidence,ymin,xmin,ymax,xmax,class_NEG,class_Trophozoite,class_WBC
0,1,712,1241,737,1270,0,1,0
1,1,558,1566,600,1604,0,1,0
2,1,1317,2788,1448,2914,0,1,0
3,1,925,1744,1041,1823,0,1,0
4,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
23525,1,2336,2441,2537,2647,0,0,1
23526,1,486,713,543,770,0,0,1
23527,1,456,497,486,535,0,1,0
23528,1,2974,1039,3094,1165,0,1,0


In [11]:
df["confidence"].nunique()

1

In [12]:
X=df.drop(columns=["class_NEG","class_Trophozoite","class_WBC"])
Y=df[["class_NEG","class_Trophozoite","class_WBC"]]

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [16]:
random_reg=RandomForestRegressor()
random_reg.fit(X_train,Y_train)
random_reg.score(X_test,Y_test)

0.8650422901479954

# # Deep learning RNN

In [53]:
from keras.layers import Dense,Flatten,LSTM,GRU,SimpleRNN,BatchNormalization,Dropout,Input,ReLU,TimeDistributed,Layer
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [22]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1
)

In [23]:
X_train.shape

(18824, 5)

In [36]:
X_train_reshape=X_train.values
X_train_reshape=X_train_reshape.reshape(X_train.shape[0],1,X_train.shape[1])
X_test_reshape=X_test.values
X_test_reshape=X_test_reshape.reshape(X_test.shape[0],1,X_test.shape[1])
X_train_reshape.shape

(18824, 1, 5)

In [39]:
X_test_reshape.shape

(4706, 1, 5)

In [43]:
inputs=Input(shape=(1,5))
S=SimpleRNN(32,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(inputs)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=SimpleRNN(64,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
S=ReLU()(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=LSTM(128,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
S=ReLU()(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=LSTM(256,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
S=ReLU()(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(556,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
S=ReLU()(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(1024,return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
S=ReLU()(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=Flatten()(S)
D=Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2())(S)
D=Dropout(0.5)(D)
D=BatchNormalization()(D)
D=Dense(32,activation="relu")(D)
D=Dropout(0.5)(D)
D=BatchNormalization()(D)
outputs=Dense(3,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_10 (SimpleRNN)       │ (None, 1, 32)          │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 1, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_11 (SimpleRNN)       │ (None, 1, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_20 (ReLU)                 │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 1, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 1, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 1, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 1, 256)         │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ (None, 1, 556)         │     1,357,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 1, 556)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 1, 556)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 1, 556)         │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 1, 1024)        │     4,859,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 1, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 6,794,539 (25.92 MB)

 Trainable params: 6,790,227 (25.90 MB)

 Non-trainable params: 4,312 (16.84 KB)

In [44]:
model.fit(X_train_reshape,Y_train,epochs=50,batch_size=32,validation_data=(X_test_reshape,Y_test),callbacks=[early_stopping])

Epoch 1/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - accuracy: 0.4885 - loss: 59.7083 - val_accuracy: 0.6766 - val_loss: 8.8061
Epoch 2/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 46s 78ms/step - accuracy: 0.6650 - loss: 6.5122 - val_accuracy: 0.7036 - val_loss: 2.5853
Epoch 3/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 45s 77ms/step - accuracy: 0.6748 - loss: 2.2393 - val_accuracy: 0.7034 - val_loss: 1.4102
Epoch 4/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 44s 75ms/step - accuracy: 0.6666 - loss: 1.3782 - val_accuracy: 0.7034 - val_loss: 1.0508
Epoch 5/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 46s 79ms/step - accuracy: 0.6824 - loss: 1.0858 - val_accuracy: 0.7034 - val_loss: 0.9870
Epoch 6/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - accuracy: 0.6744 - loss: 1.0247 - val_accuracy: 0.7034 - val_loss: 0.9427
Epoch 7/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 51s 87ms/step - accuracy: 0.6854 - loss: 0.9877 - val_accuracy: 0.7250 - val_loss: 0.9235
Epoch 8/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 86s 94ms/step - accuracy: 0.6976 - loss: 0.9895 -

# # Quantum LSTM

In [45]:
!pip install cirq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.6 MB

In [47]:
import tensorflow as tf
import cirq


In [78]:
def build_quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(len(data))]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*np.pi).on(qubit[i]))
        circuit.append(cirq.ry(value*np.pi).on(qubit[i]))
        circuit.append(cirq.rz(value*np.pi).on(qubit[i]))
    circuit.append([cirq.CNOT(qubit[i],qubit[i+1]) for i in range(len(data)-1)])
    circuit.append([cirq.measure(qubit[i],key=f"qubit_{i}") for i in range(len(data))])
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for input_data in data:
        circuit,qubit=build_quantum_circuit(input_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        for i in range(len(input_data)):
            measure0=result.measurements[f'qubit_{i}'].flatten()[0]
            results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        inputs_flat=tf.reshape(inputs,[-1,inputs.shape[-1]])
        output=tf.numpy_function(run_quantum_circuit,[inputs_flat],tf.float32)
        output.set_shape(inputs.shape)
        return output
model2=Sequential([
    QuantumLayer(),
    Flatten(),
    
    Dense(32,activation="relu"),
    Dense(3,activation="softmax")
])
model2.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
model2.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ quantum_layer_12 (QuantumLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [79]:
model2.fit(X_train_reshape,Y_train,epochs=50,batch_size=32,validation_data=(X_test_reshape,Y_test),callbacks=[early_stopping])

Epoch 1/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 132s 222ms/step - accuracy: 0.6603 - loss: 1.0204 - val_accuracy: 0.6766 - val_loss: 0.8572
Epoch 2/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 131s 222ms/step - accuracy: 0.6634 - loss: 0.8408 - val_accuracy: 0.6766 - val_loss: 0.7769
Epoch 3/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 129s 218ms/step - accuracy: 0.6720 - loss: 0.7738 - val_accuracy: 0.6766 - val_loss: 0.7467
Epoch 4/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 131s 222ms/step - accuracy: 0.6741 - loss: 0.7490 - val_accuracy: 0.6766 - val_loss: 0.7337
Epoch 5/50
589/589 ━━━━━━━━━━━━━━━━━━━━ 129s 219ms/step - accuracy: 0.6764 - loss: 0.7360 - val_accuracy: 0.6766 - val_loss: 0.7277
Epoch 5: early stopping
